In [3]:
from pyroutelib3 import Router, Datastore # Import the router
router = Router("cycle", "C:/Users/pfrod/OneDrive/Documents/1A mines de paris/MIG/carte_test.osm") # Initialise it


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/pfrod/OneDrive/Documents/1A mines de paris/MIG/carte_test.osm'

In [ ]:
start = router.findNode(43.7006971, 7.2587751) # Find start and end nodes
end = router.findNode(43.700561, 7.2599291)

In [ ]:
status, route = router.doRoute(start, end) # Find the route - a list of OSM nodes


In [ ]:
status

In [ ]:
route

In [ ]:
list(map(router.nodeLatLon, route))

In [2]:
surface = 1  #récupérer la carte ensoleillement et altitude avec les points dessus 
(pas,pastemp) = (7000000,1)
(xo,yo,to) = (3,2,0)
vitesse = 20/3.6
import numpy as np
puiscarac = 140 #typiquement la puissance moyenne restituée par un panneau sur un triporteur
cartalt = np.random.randint(50, size=(80,80))
cartensol =np.random.randint(50, size=(80,80, 5))
from pyproj import Transformer
transformer_to_lamb= Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
transformer_to_lat_long=Transformer.from_crs( "EPSG:2154","EPSG:4326", always_xy=True)
L=[(43.7006972, 7.2587752),
 (43.7006331, 7.258767),
 (43.700484, 7.2587479),
 (43.7004658, 7.2587252),
 (43.7004336, 7.258622),
 (43.7004036, 7.2586002),
 (43.6996616, 7.2584458),
 (43.6995896, 7.2584263),
 (43.6995951, 7.259249),
 (43.6995986, 7.2597091),
 (43.700563, 7.2599293)]
G=[transformer_to_lamb.transform(L[i][0],L[i][1]) for i in range(len(L))]
class Point :
    def __init__(self, lat, lon) :
        self.latitude = lat
        self.longitude = lon
        self.id = router.findNode(lat, lon)
        self.x = transformer_to_lamb.transform(lat,lon)[0]
        self.y = transformer_to_lamb.transform(lat,lon)[1]
        self.alti = cartalt[round((self.x-xo) / pas)][round((self.y-yo) / pas)]
    
    def __repr__(self) :
        return (f"point de latitude {self.latitude}, de longitude {self.longitude}, d'altitude {self.alti}, x={self.x}, y={self.y} ")

class DeliveryPoint(Point):
    def __init__(self, lat, lon, t1, t2):
        Point.__init__(self, lat, lon)
        self.t1 = t1
        self.t2 = t2
    def __repr__(self):
        return (Point.__repr__(self)+f"il veut être livré entre {self.t1} et {self.t2}")    
        
def ensol(point, t) :
    pix_x = round((point.x-xo) / pas)
    pix_y = round((point.y-yo) / pas)
    pix_t = round((t-to) / pastemp)
    return cartensol[pix_x, pix_y, pix_t] * surface


def distroute(point1, point2):
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l = 0
    for i in range(len(route)-1):
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += router.distance((point1.latitude, point1.longitude),(point2.latitude, point2.longitude))
    return l * 1000


def enersol(point1, point2, t):
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l=0
    for i in range(len(route)-1) :
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += router.distance((point1.latitude, point1.longitude),(point2.latitude, point2.longitude)) * ensol(point1, t) * vitesse * 1000
    return l 
def gaintottrajet(point1, point2, t):
    return enersol(point1, point2, t)-depense(point1, point2)

def deniv(point1, point2) :
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l = 0
    for i in range(len(route)-1):
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += max(0,point2.alti-point1.alti)
    return l


def graph(nodeslist, t, a, b, c) :#nodeslist est la liste des (x,y) des points de livraison 
    if not(( 0 <= a <= 1 ) and ( 0 <= b <= 1 ) and ( 0 <= c <= 1 )):
        raise ValueError("Les paramètres sont entre 0 et 1")
    for i in range(len(nodeslist)) :
        nodeslist[i] = Point(transformer_to_lat_long.transform(nodeslist[i][0],nodeslist[i][1])[0],transformer_to_lat_long.transform(nodeslist[i][0],nodeslist[i][1])[1])
    def poids(point1,point2,t):
        return a * distroute(point1, point2) + b * 100 * deniv(point1, point2) - c * vitesse / puiscarac * gaintottrajet(point1, point2, t)
    graphe = {}
    for s in nodeslist :
        graphe[s] = {}
        for j in nodeslist : 
            if j!=s:
                graphe[s][j] = poids(s, j, t)
    return graphe


    
DeliveryPoint(2,3,5,7)
#graph(G, 1, 0.5, 0.2, 0.7)


NameError: name 'router' is not defined

In [ ]:
router.rnodes[311258859]
print(router.routing[273485738])
print(distroute(Point(router.rnodes[311258859][0],router.rnodes[311258859][1]),Point(router.rnodes[564661114][0],router.rnodes[564661114][1])))
point1=Point(2.0,3.2)
print(point1)

Cette fonction permet, en prenant en entrée la map de pyroute avec son système d'itinéraire ainsi que les données d'ensoleillement et d'altitude sous forme de matrice, de ressortir un graphe dont les poids des arêtes dépendent de l'ensoleillement, le dénivelé, et la distance. Les matrices d'entrée sont données avec une case par pixel, et on a une relation directe entre taille du pixel et longueur en m. En revanche, la map pyroute permet de repérer les points à la fois avec des coordonnées lat lon et des identifiants, en sachant que si on prend une coordonnée donnée, le programme va calculer tout avec l'identifiant du point le plus proche du point d'entrée. C'est pourquoi les arguments d'un attribut point donné sont ses coordonnées en lat lon. On peut tout de même transcrire lat lon en m et inversement. 

In [ ]:
surface = 1  #récupérer la carte ensoleillement et altitude avec les points dessus 
(pas,pastemp) = (7000000,1)
(xo,yo,to) = (3,2,0)
vitesse = 20/3.6
batterietot = 500 #1500 si il est vener
import numpy as np
puiscarac = 140 #typiquement la puissance moyenne restituée par un panneau sur un triporteur
cartalt = np.random.randint(50, size=(80,80))
from pyproj import Transformer
transformer_to_lamb= Transformer.from_crs("EPSG:4326", "EPSG:2154", always_xy=True)
transformer_to_lat_long=Transformer.from_crs( "EPSG:2154","EPSG:4326", always_xy=True)
L=[(43.7006972, 7.2587752),
 (43.7006331, 7.258767),
 (43.700484, 7.2587479),
 (43.7004658, 7.2587252),
 (43.7004336, 7.258622),
 (43.7004036, 7.2586002),
 (43.6996616, 7.2584458),
 (43.6995896, 7.2584263),
 (43.6995951, 7.259249),
 (43.6995986, 7.2597091),
 (43.700563, 7.2599293)]
G=[transformer_to_lamb.transform(L[i][0],L[i][1]) for i in range(len(L))]
class Point :
    def __init__(self, lat, lon) :
        self.latitude = lat
        self.longitude = lon
        self.id = router.findNode(lat, lon)
        self.x = transformer_to_lamb.transform(lat,lon)[0]
        self.y = transformer_to_lamb.transform(lat,lon)[1]
        self.alti = cartalt[round((self.x-xo) / pas)][round((self.y-yo) / pas)]
    
    def __repr__(self) :
        return (f"point de latitude {self.latitude}, de longitude {self.longitude}, d'altitude {self.alti}, x={self.x}, y={self.y} ")

class DeliveryPoint(Point):
    def __init__(self, lat, lon, t1, t2):
        Point.__init__(self, lat, lon)
        self.t1 = t1
        self.t2 = t2
    def __repr__(self):
        return (Point.__repr__(self)+f"il veut être livré entre {self.t1} et {self.t2}")    
class Bornes(Point):
    def __init__(self, lat, lon, priseslibres, charges):
        Point.__init__(self, lat, lon)
        self.priseslibres = priseslibres #tableau de booleen: à chaque case du tableau est associée une prise
        self.charges = charges #tableau en Wh des charges de chacune des batteries 
    def reserv(self):
        for i in range len(priseslibres):
            if priseslibres[i] & :# ajouter condition de choix fixée par romain et Jeremy
                priseslibres[i] = False #on choisit la prise 
                return i
        return None #ie c'est tout réservé déjà
    def dereserv(self, i, chargetriporteuravant):
        priseslibres[i] = True 
        charges[i] = chargetriporteuravant #ie on a remplacé la batterie par la notre
class Poids:
    def __init__(self, energienecess, t, faisable):
        self.energie = energienecess
        self.duree = t
        self.faisable = faisable #un booléen 
def distroute(point1, point2):
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l = 0
    for i in range(len(route)-1):
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += router.distance((point1.latitude, point1.longitude),(point2.latitude, point2.longitude))
    return l * 1000



def batteriefintrajet(point1, point2, t):
    return batterieinit-depense(point1, point2)

def deniv(point1, point2) :
    start = point1.id # Find start and end nodes
    end = point2.id
    status, route = router.doRoute(start, end)
    l = 0
    for i in range(len(route)-1):
        point1 = Point(router.rnodes[route[i]][0],router.rnodes[route[i]][1]) 
        point2 = Point(router.rnodes[route[i+1]][0],router.rnodes[route[i+1]][1])
        l += max(0,point2.alti-point1.alti)
    return l


def graph(nodeslist, t, a, b, c) :#nodeslist est la liste des (x,y) des points de livraison 
    if not(( 0 <= a <= 1 ) and ( 0 <= b <= 1 ) and ( 0 <= c <= 1 )):
        raise ValueError("Les paramètres sont entre 0 et 1")
    for i in range(len(nodeslist)) :
        nodeslist[i] = Point(transformer_to_lat_long.transform(nodeslist[i][0],nodeslist[i][1])[0],transformer_to_lat_long.transform(nodeslist[i][0],nodeslist[i][1])[1])
    def poids(point1,point2,t):
        return (distroute(point1, point2), puissmax(point1, point2)) a * distroute(point1, point2) + b * 100 * deniv(point1, point2) - c * vitesse / puiscarac * gaintottrajet(point1, point2, t)
    graphe = {}
    for s in nodeslist :
        graphe[s] = {}
        for j in nodeslist : 
            if j!=s:
                graphe[s][j] = poids(s, j, t)
    return graphe



Ces programmes tentent cependant de faire correspondre les points de la carte reprise par Pyroute, et de la carte d'altitude et d'ensoleillement fournie par Juliette, et on rencontre des problèmes pour les utiliser de manière synergique. De plus, si on optimise non pas la distance parcourue mais l'énergie pour parcourir le graphe des clients, le chemin utilisé pour aller chez ces différents clients est lui fixé par Pyroute, et donc cherche uniquement à minimiser la distance. Ce n'est pas cohérent. 

In [ ]:
#on recupere la carte sous forme d'ensemble de liste de points coor avec lat et long, ainsi qu'une matrice de Booléens route
#tq route[i][j] = 1 si il y a une route dans le bon sens entre les deux, 0 sinon. Elle envoie aussi une fonction altitude 
#Arthur m'envoie ensuite le programme qui prend une suite de quintuplets et qui renvoie la dépense énergétique et la puissance maximale. 
#elle devrait aussi m'envoyer la liste des Booléens tq piet[i][j] = 1 si il y a une zone piétonne 0 sinon 
vitessepiet = 10/3.6
vitesse = 25/3.6
piet = []
piet_coor = {Point(coor[i][0], coor[i][1]):{Point(coor[j][0], coor[j][1]):piet[i][j] for j in range(len(piet[i]))}for i in range(len(piet))}
#on a piet_coor[point1][point2] qui n'est défini que si point1 et point2 sont reliés

def distance_euc(L, N):
    return sqrt((L[0]-N[0])**2+(L[1]-N[1])**2)


def speed(piet, i, j):
    if piet[i][j] == 1:
        return vitessepiet
    else :
        return vitesse


def grosgraph(coor, route, piet):
    grosgraphe = {} ; graphvit = {}
    for i in range(len(coor)) : 
        p = Point(coor[i][0],coor[i][1])
        grosgraphe[p] = {} ; graphvit[p] = {}
        for j in range(len(coor[i])) :
            q = Point(coor[j][0],coor[j][1])
            grosgraphe[p][q] = calcul_energy((distance_euc(coor[i],coor[j]), altitude(coor[i][0], coor[i][1]), altitude(coor[j][0], coor[j][1]), speed(piet, i, j), []))[0] + puisempl * distance_euc(coor[i],coor[j])/speed(piet, i, j)
    return grosgraphe
                

def recurs(graphe, x, s, t, P): 
    if P[x] == s:
        M.append(s)
        M.reverse()
    if P[x] != s :
        M.append(P[x])
        recurs(graphe, P[x], M, s, t, P)
        

def djikstra(graphe,etape,fin,visites,dist,P,depart):
    M={}
    if etape == fin : 
        return (dist[fin], recurs(graphe, fin, M, depart, fin, P))
    if len(visites) == 0:
        dist[etape] = 0
    for voisin in graphe[etape].keys():
        if voisin not in visites:
            if dist[voisin]>dist[etape] + graphe[etape][voisin]:
                dist[voisin] = dist[etape]+graphe[etape][voisin]
                P[voisin] = etape
    visites.append(etape)
    distance = float('inf')
    for x in graphe.keys():
        if x not in visites : 
            if dist[x]<distance : 
                x_min = x
                distance = dist[x]
    return djikstra(graphe, x_min, fin, visites, dist, P, depart)
    

def path_clients(coor, route, piet, nodeslist, bornes, elp):
    M = {}
    for s in nodeslist.union(bornes.union(elp)):
        M[s] = {}
        for t in nodeslist.union(bornes.union(elp)):
            M[s][t] = djikstra(grosgraphe(coor, route, piet), s, t, [], graph[s], {}, s)[1]
    return M #ca renvoie un graphe de liste avec les listes de point liant les point du graphe


def temps(coor, route, piet, piet_coor, depart, arrivee):
    t = 0; L = djikstra(grosgraphe(coor, route, piet), depart, arrivee, [], graph[depart], {}, depart)[1]
    for i in range(len(L)-1):
        t += distance_euc(L[i],L[i+1])/speed(piet_coor, L[i], L[i+1])
    return t #à 


def trouvpoint(coor, route, piet, piet_coor, depart, arrivee, tdepuisdep):
    ttot = temps(coor, route, piet, piet_coor, depart, arrivee) ; L = djikstra(grosgraphe(coor, route, piet), depart, arrivee, [], graph[depart], {}, depart)[1]; i = 0
    while temps(coor, route, piet, piet_coor, depart, L[i])< tdepuisdep : 
        i+=1
    return L[i] #c'est de la classe point



def graph(coor, route, piet, nodeslist, bornes, elp):
    sousgraphe = {}
    for s in nodeslist.union(bornes.union(elp)) : 
        sousgraphe[s] = {} 
        for t in nodeslist.union(bornes.union(elp)) : 
            sousgraphe[s][t] = Poids(djikstra(grosgraphe(coor, route, piet), s, t, [], graph[s], {}, s)[0], temps(coor, route, piet, piet_coor, depart, arrivee),type(calcultot(coordonnées à fixer par arthur)) != str)
    return sousgraphe
#attention les bornes et les points de livraison sont seulement des points ici, pour les différencier il faut avoir la liste des bornes                                 
